# Connect to the API

In [147]:
import pprint as pp
import requests
from opensearchpy import OpenSearch
from opensearchpy import helpers

host = 'api.novasearch.org'
port = 443
user = 'user201' # Add your user name here.
password = 'Lrr1531' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

# Delete an index

In [148]:
response = client.indices.delete(
        index = user
    )
print(response)

{'acknowledged': True}


# Index creation

In [149]:
index_body = {
    "settings": {
    "index": {
      "number_of_replicas":0,
      "number_of_shards":4,
      #"refresh_interval":"-1", -- it's crashing the search
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
    "mappings": {
        "properties": {
            "title": {
                "type": "text"
            },
            "description": {
                "type": "text"
            },
            "title_embedding": {
                "type":"knn_vector",
            "dimension": 768,
            "method":{
               "name":"hnsw",
               "space_type":"innerproduct",
               "engine":"faiss",
               "parameters":{
                  "ef_construction":256,
                  "m":48
               }
            },
            },
            "ingredients": {
                "type": "keyword"
            },
            "duration": {
                "type": "integer"
            },
            "steps": {
                "type": "text"
            },
        
        }
    }
}

if client.indices.exists(index=index_name):
    print("Index already existed. You may force the new mappings.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user201'}


# Documents creation

In [168]:
import json as json
import doc


with open("./recipes/recipes_data.json", "r") as read_file:
    data = json.load(read_file)

titles = [data[key]['displayName'] for key in data] 


doc.index_document(client, index_name, data)




updated: How To Make Chicken Parmesan
updated: How to Make Pesto
updated: How To Make Corn Tortillas From Scratch
updated: How To Make Elote (Mexican Street Corn)
updated: How To Make Macarons
updated: How To Make Meringue
updated: How To Make Handmade Pasta
updated: How to Make Perfect Polenta
updated: How To Make Miso Soup
updated: How to Make Perfect Guacamole
updated: How to Cook Trout
updated: How to Cook a Turkey
updated: How to Cook: Boiled Eggs
updated: How To Cook A Perfect Risotto
updated: How to Cook: Crispy Tofu
updated: How to Cook: Brown Rice
updated: How To Cook Salmon in the Oven
updated: How To Cook Brown Rice
updated: How To Cook Filet Mignon
updated: How To Cook Lentils on the Stove
updated: Sylvia's World Famous Talked About Spareribs
updated: Holiday Eggnog
updated: Holiday Pizza
updated: Holiday Salad
updated: Holiday Seafood Pot
updated: Holiday Chicken Salad
updated: Holiday milk punch
updated: Holiday Manhattan
updated: Snickerdoodles I
updated: Holiday Pumpkin

AttributeError: module 'doc' has no attribute 'index_embedding'

AttributeError: module 'doc' has no attribute 'embeddings_index'

#Searches and embeddings

In [152]:
data['993']

{'displayName': 'Full House Martini',
 'description': None,
 'canonicalName': 'Full House Martini',
 'prepTimeMinutes': 5,
 'cookTimeMinutes': None,
 'totalTimeMinutes': 5,
 'cookingMethod': None,
 'difficultyLevel': None,
 'images': [{'url': 'https://m.media-amazon.com/images/S/alexa-kitchen-msa-na-prod/recipes/foodnetwork/f38926516590827e25b36d00327c0d976de77bc4876c4c2b45666b9a7cfe4481.jpg',
   'hdURL': None,
   'fourKURL': None,
   'description': None,
   'type': 'UNKNOWN'}],
 'videos': [{'providerId': None,
   'title': None,
   'url': 'https://d3891lgtxgmxvq.cloudfront.net/recipes/foodnetwork/50edcb738ae2e6e885eb4a19c789fe37226f6c62c8d71975ef78eea52d7f434c.m3u8',
   'mobileUrl': 'https://d3891lgtxgmxvq.cloudfront.net/recipes/foodnetwork/50edcb738ae2e6e885eb4a19c789fe37226f6c62c8d71975ef78eea52d7f434c.mp4',
   'hdURL': None,
   'fourKURL': None,
   'description': None,
   'type': 'EPISODE'}],
 'tools': [],
 'cuisines': None,
 'meals': None,
 'courses': None,
 'occasions': None,
 'di

In [153]:
qtxt = "martini"

query_bm25 = {
  'size': 5,
  '_source': ['title', 'description'],
  'query': {
    'multi_match': {
      'query': qtxt,
      'fields': ['title']  #some recipes don't have a description
    }
  }
}

response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response['hits']['hits'])
print(response['hits']['total']['value'])




Search results:
[{'_id': '994',
  '_index': 'user201',
  '_score': 5.902687,
  '_source': {'description': None, 'title': 'Full House Martini'},
  '_type': '_doc'},
 {'_id': '682',
  '_index': 'user201',
  '_score': 5.23184,
  '_source': {'description': None, 'title': 'Vodka Martini'},
  '_type': '_doc'},
 {'_id': '747',
  '_index': 'user201',
  '_score': 4.813482,
  '_source': {'description': None, 'title': 'Flat white martini'},
  '_type': '_doc'},
 {'_id': '685',
  '_index': 'user201',
  '_score': 4.6468973,
  '_source': {'description': None, 'title': 'Dirty Martini (Vodka)'},
  '_type': '_doc'},
 {'_id': '789',
  '_index': 'user201',
  '_score': 4.6468973,
  '_source': {'description': None, 'title': 'Cotton Candy Martini'},
  '_type': '_doc'}]
7


In [154]:
qtxt = 'strawberry'

query_bm25 = {
  'size': 5,
  '_source': ['title', 'description'],
  'query': {
    'multi_match': {
      'query': qtxt,
      'fields': ['title']  #some recipes don't have a description
    }
  }
}

response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response['hits']['hits'])
# number of results
print(response['hits']['total']['value'])




Search results:
[{'_id': '167',
  '_index': 'user201',
  '_score': 5.3236856,
  '_source': {'description': None, 'title': 'Chocolate Strawberry Trees'},
  '_type': '_doc'},
 {'_id': '469',
  '_index': 'user201',
  '_score': 5.3236856,
  '_source': {'description': 'Strawberry in two forms infuses this layer cake '
                             'with beautiful color and natural flavor.',
              'title': 'Double Strawberry Cake'},
  '_type': '_doc'},
 {'_id': '401',
  '_index': 'user201',
  '_score': 5.056594,
  '_source': {'description': None,
              'title': 'Strawberry Sauce for Strawberry Shortcake'},
  '_type': '_doc'},
 {'_id': '519',
  '_index': 'user201',
  '_score': 4.519971,
  '_source': {'description': None, 'title': 'Strawberry Cake from Scratch'},
  '_type': '_doc'},
 {'_id': '500',
  '_index': 'user201',
  '_score': 4.3898854,
  '_source': {'description': None, 'title': 'Doughnut Strawberry Shortcake'},
  '_type': '_doc'}]
12


### Boolean queries

In [155]:
qtxt = "Ingredients for chicken marsala"

query_bm25 = {
    "size": 5,
    "_source": ["title", "ingredients"],
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "description": "chicken marsala"
                    }
                }
            ],
            "should": [
                {
                    "multi_match": {
                        "query": qtxt,
                        "fields": ["title", "description"]
                    }
                }
            ]
        }
    }
}



response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)




Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '407',
                    '_index': 'user201',
                    '_score': 12.2271,
                    '_source': {'ingredients': ['butter',
                                                'oil',
                                                'chicken breast',
                                                'salt',
                                                'flour',
                                                'cream',
                                                'chicken broth',
                                                'Alcoholic beverage, wine, '
                                                'table, all',
                                                'garlic',
                                                'mushrooms',
                                                'butter',
                                                'lemon',
            

### Dual encoders


In [156]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pickle


#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")


doc_emb = encode(titles)

with open('embeddings.pickle', 'wb') as f:
    pickle.dump(doc_emb, f)











AttributeError: module 'doc' has no attribute 'index_embedding'

In [ ]:
# Compute the query embedding
query = "cake"
query_emb = encode(query)

query_denc = {
  'size': 5,
  '_source': ['title', 'description'],
   "query": {
        "knn": {
          "title_embedding": {
            "vector": query_emb[0].numpy(),
            "k": 2
          }
        }
      }
}

response = client.search(
    body = query_denc,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)


Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': 'Semsr44B7xTIhwxmSFcg',
                    '_index': 'user201',
                    '_score': 1.7172348,
                    '_source': {'title': 'My Birthday Cake'},
                    '_type': '_doc'},
                   {'_id': 's-msr44B7xTIhwxmrVjA',
                    '_index': 'user201',
                    '_score': 1.689884,
                    '_source': {'title': "A Chocolate Cake That's Got It All"},
                    '_type': '_doc'},
                   {'_id': 'Jemsr44B7xTIhwxmhViq',
                    '_index': 'user201',
                    '_score': 1.6858522,
                    '_source': {'title': 'Christmas Cake Cookies'},
                    '_type': '_doc'},
                   {'_id': 'Y-msr44B7xTIhwxml1g0',
                    '_index': 'user201',
                    '_score': 1.6794477,
                    '_source': {'title': 'Chocolate Chip C

In [ ]:
'''
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

#Define your train examples. You need more than just two examples to get good results from the model im using



#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
'''

"\nfrom sentence_transformers import SentenceTransformer, InputExample, losses\nfrom torch.utils.data import DataLoader\n\n#Define the model. Either from scratch of by loading a pre-trained model\nmodel = SentenceTransformer('distilbert-base-nli-mean-tokens')\n\n#Define your train examples. You need more than just two examples to get good results from the model im using\n\n\n\n#Define your train dataset, the dataloader and the train loss\ntrain_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)\ntrain_loss = losses.CosineSimilarityLoss(model)\n\n#Tune the model\nmodel.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)\n"